# 🔧 PASO 1 – Preparar dataset

In [42]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Crear función de preparación con lags y clase
def preparar_dataset_para_regresion(df):
    df = df.sort_values(['product_id', 'periodo'])
    
    # Generar lags tn_1 a tn_11 por producto
    for i in range(1, 12):
        df[f'tn_{i}'] = df.groupby('product_id')['tn'].shift(i)
    

    

    # Crear campo "clase" con tn en periodo+2
    df['clase'] = df.groupby('product_id')['tn'].shift(-2)
    
    return df

In [43]:
df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
df = df.groupby(['periodo', 'product_id']).agg({'tn' : 'sum'}).reset_index() # Agrupar por periodo y producto
df_prep = preparar_dataset_para_regresion(df)  # Preparar el dataset
df_prep

,periodo,product_id,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,clase
0,201701,20001,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1303.35771
785,201702,20001,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1069.96130
1566,201703,20001,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1502.20132
2352,201704,20001,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1520.06539
3136,201705,20001,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1030.67391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,201701,21295,0.00699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6435,201708,21296,0.00651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
784,201701,21297,0.00579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6436,201708,21298,0.00573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 📚 PASO 2 – Entrenar regresión lineal

In [44]:
import pandas as pd
import numpy as np

def get_productos_36_meses(magicos, n_random=30, seed=42):
    df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
    df = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()  # Agrupar por periodo y producto

    # Paso 1: calcular la cantidad de registros por product_id
    conteo = df.groupby('product_id').size()

    # Paso 2: quedarte solo con los que tienen 36 registros
    productos_36_meses = conteo[conteo == 36].index.tolist()

    # Paso 3: eliminar productos mágicos
    productos_restantes = list(set(productos_36_meses) - set(magicos))

    # Paso 4: elegir aleatoriamente N productos distintos a los mágicos
    np.random.seed(seed)
    productos_random = np.random.choice(productos_restantes, size=n_random, replace=False).tolist()

    # Paso 5: concatenar mágicos + aleatorios
    productos_finales = magicos + productos_random

    # Paso 6: filtrar el DataFrame con esos productos
    df_filtrado = df[df['product_id'].isin(productos_finales)].copy()

    return df_filtrado


In [53]:
# Lista de productos mágicos
magicos = [
    20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
    20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046, 20049,
    20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
    20193, 20320, 20532, 20612, 20637, 20807, 20838
]


get_productos_36_meses(magicos)['product_id'].unique()

array([20001, 20002, 20003, 20006, 20008, 20010, 20011, 20017, 20018,
       20019, 20021, 20023, 20026, 20028, 20035, 20037, 20039, 20042,
       20044, 20045, 20046, 20051, 20052, 20053, 20055, 20086, 20109,
       20114, 20118, 20158, 20167, 20176, 20180, 20181, 20193, 20272,
       20320, 20321, 20358, 20361, 20400, 20410, 20416, 20434, 20454,
       20465, 20466, 20501, 20532, 20568, 20612, 20637, 20672, 20680,
       20684, 20685, 20807, 20820, 20838, 20864, 20986, 21218, 20049])

In [52]:
productos = pd.read_csv('../../data/raw/tb_productos.csv', sep='\t')  # Cargar el dataset
productos

,cat1,cat2,cat3,brand,sku_size,product_id
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180
2,HC,ROPA LAVADO,Liquido,LIMPIEX,120,20332
3,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20288
...,...,...,...,...,...,...
1257,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,12,20240
1258,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,6,20285
1259,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,2,20259
1260,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,12,20142


In [54]:
productos[productos['product_id'].isin(magicos)]

,cat1,cat2,cat3,brand,sku_size,product_id
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180
9,HC,ROPA MANCHAS,Ladrillo,LIMPIEX,200,20039
10,HC,VAJILLA,Opaco,LIMPIEX,750,20017
11,HC,VAJILLA,Opaco,LIMPIEX,750,20008
12,HC,VAJILLA,Cristalino,LIMPIEX,750,20018
14,HC,VAJILLA,Cristalino,LIMPIEX,750,20006
21,HC,ROPA LAVADO,Liquido,LIMPIEX,800,20026
22,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,20002
36,HC,ROPA LAVADO,Polvo,LIMPIEX,400,20010
37,HC,ROPA LAVADO,Polvo,LIMPIEX,400,20010


In [46]:
# Entrenar regresión lineal en periodo base con productos mágicos
def entrenar_regresion_lineal(df, periodo_base, magicos):
    columnas_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
    
    # Filtrar registros del periodo base y productos mágicos
    df_train = df[(df['periodo'].isin(periodo_base)) & (df['product_id'].isin(magicos))].copy()
    
    # Eliminar filas con datos faltantes
    df_train = df_train.dropna(subset=columnas_tn + ['clase'])

    # Entrenar modelo
    X = df_train[columnas_tn]
    y = df_train['clase']
    model = LinearRegression()
    model.fit(X, y)

    # Mostrar coeficientes
    coef = pd.Series([model.intercept_] + model.coef_.tolist(), index=['intercept'] + columnas_tn)
    return model, coef


# 🔮 PASO 3 – Predecir con el modelo entrenado

In [47]:
def productos_con_historia():
    df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
    df = df.groupby(['periodo', 'product_id']).agg({'tn' : 'sum'}).reset_index() # Agrupar por periodo y producto
    productos = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')  # Cargar productos
    df = df[df['product_id'].isin(productos['product_id'].unique())].copy()  # Filtrar productos a predecir

    contador = df[(df['periodo']>=201901) & (df['periodo']<=201912) ] 
    contador = contador.groupby('product_id').size().reset_index(name='count')
    product_id = contador[contador['count']==12]['product_id'].unique()

    return product_id

In [48]:
# Predecir para un periodo futuro
def predecir_regresion(model):
    df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
    df = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()
    productos = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
    df = df[df['product_id'].isin(productos['product_id'].unique())].copy()
    df = df[(df['periodo'] >= 201901) & (df['periodo'] <= 201912)]
    df = df[df['product_id'].isin(productos_con_historia())]
    
    df = df.sort_values(['product_id', 'periodo'])

    # Generar lags tn_1 a tn_11 por producto
    for i in range(1, 12):
        df[f'tn_{i}'] = df.groupby('product_id')['tn'].shift(i)

    

    columnas_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)] 

    # Filtrar periodo de predicción
    df_pred = df[df['periodo'] == 201912].copy()

    # Asegurar que no haya nulls
    df_pred = df_pred.dropna(subset=columnas_tn)

    # Predecir
    df_pred['pred'] = model.predict(df_pred[columnas_tn])

    return df_pred


def predecir_no_completos():
    
    df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
    df = df.groupby(['periodo', 'product_id']).agg({'tn' : 'sum'}).reset_index() # Agrupar por periodo y producto
    productos = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')  # Cargar productos
    df = df[df['product_id'].isin(productos['product_id'].unique())].copy()  # Filtrar productos a predecir
    df = df[(df['periodo']>=201901) & (df['periodo']<=201912) ] 
    df = df[~df['product_id'].isin(productos_con_historia())]
    
    df = df.sort_values(['product_id', 'periodo'])
    
    df = df.groupby('product_id').agg({'tn':'mean'}).reset_index()  # Tomar los últimos 12 meses de cada producto
    
    return df

In [49]:
# prods = predecir_regresion(modelo)
# prods.sort_values(['product_id','periodo'], inplace=True)
# prods

# ▶️ PASO 4 – Ejecutar todo el flujo

In [ ]:
# Paso A: preparar dataset (con tus datos originales en df)
# df_prep = preparar_dataset_para_regresion(df)

magicos_aux = get_productos_36_meses(magicos,n_random=0)['product_id'].unique()
meses = [201812]
# Paso B: entrenar el modelo con periodo 201812
modelo, coeficientes = entrenar_regresion_lineal(df_prep, periodo_base=meses, magicos=magicos_aux)
print("Coeficientes encontrados:")
print(coeficientes)

# Paso C: predecir para registros del periodo 201912 → objetivo: predecir 202002
df_pred_con_historia = predecir_regresion(modelo)
df_pred_con_historia = df_pred_con_historia[['product_id', 'pred']].rename(columns={'pred': 'tn'})

df_pred_sin_historia = predecir_no_completos()

df_pred = pd.concat([df_pred_con_historia, df_pred_sin_historia], ignore_index=True)

# Paso D: ver primeras predicciones
df_pred

Coeficientes encontrados:
intercept    2.499441
tn          -0.457124
tn_1         0.957910
tn_2        -0.703657
tn_3         0.884039
tn_4        -0.388143
tn_5         0.951151
tn_6        -0.813920
tn_7         1.127994
tn_8        -0.446149
tn_9         0.124954
tn_10       -0.175916
tn_11       -0.503126
dtype: float64


,product_id,tn
0,20001,1649.654216
1,20002,290.412101
2,20003,366.323789
3,20004,331.030134
4,20005,715.756734
...,...,...
775,21252,0.178011
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


# 💾 PASO 5 – Exportar a CSV (opcional)

In [51]:
# # Exportar archivo para enviar a Kaggle (ajustar columnas si necesario)
df_pred[['product_id', 'tn']].to_csv("./outputs/predicciones_regresion_lineal_v7.csv", index=False)